# 资治通鉴-文本数据预处理

### 目标：一本新形式的编年体史书

任务列表：
- [x] 时间数据的译文格式保持和原文格式统一
- [x] 去除不符合文白对照格式的空行、空格
- [ ] 文本数据结构化，便于利用数据分析工具和可视化工具进行处理

In [1]:
import re
import copy
import glob
import os

from tqdm import tqdm

In [2]:
demo_f = open('chapters/001_资治通鉴第一卷(周纪).md', 'r', encoding='utf-8').read()

In [3]:
demo_f[:100]

'资治通鉴第一卷(周纪)\n\n\n\n周纪一 威烈王二十三年（戊寅、前403）\n\n周纪一 周威烈王二十三年（戊寅，公元前403年）\n\n\u3000\u3000[1]初命晋大夫魏斯、赵籍、韩虔为诸侯。\n\n\u3000\u3000[1]周威烈王姬午初次'

In [4]:
DUP_SUBTITLE_PAT = re.compile(r'\n([^\u3000\n]+)\n\n(.+\n+)')

In [5]:
re.findall(DUP_SUBTITLE_PAT, demo_f)

[('周纪一 威烈王二十三年（戊寅、前403）', '周纪一 周威烈王二十三年（戊寅，公元前403年）\n\n'),
 ('二十四年（己卯、前402）', '二十四年（己卯，公元前402年）\n\n'),
 ('安王元年（庚辰、前401）', '周安王元年（庚辰，公元前401年）\n\n'),
 ('二年（辛巳、前400）', '二年（辛巳，公元前400年）\n\n'),
 ('三年（壬午、前399）', '三年（壬午，公元前399年）\n\n'),
 ('四年（癸未、前398）', '四年（癸未，公元前398年）\n\n'),
 ('五年（甲申、前397）', '五年（甲申，公元前397年）\n\n'),
 ('六年（乙酉、前396）', '六年（乙酉，公元前396年）\n\n'),
 ('八年（丁亥、前394）', '八年（丁亥，公元前394年）\n\n'),
 ('九年（戊子、前393）', '九年（戊子，公元前393年）\n\n'),
 ('十一年（庚寅、前391）', '十一年（庚寅，公元前391年）\n\n'),
 ('十二年（辛卯、前390）', '十二年（辛卯，公元前390年）\n\n'),
 ('十三年（壬辰、前389）', '十三年（壬辰，公元前389年）\n\n'),
 ('十五年（甲午、前387）', '十五年（甲午，公元前387年）\n\n'),
 ('十六年（乙未、前386）', '十六年（乙未，公元前386年）\n\n'),
 ('十七年（丙申、前385）', '十七年（丙申，公元前385年）\n\n'),
 ('十九年（戊戌、前383）', '十九年（戊戌，公元前383年）\n\n'),
 ('二十年（己亥、前382）', '二十年（己亥，公元前382年）\n\n'),
 ('二十一年（庚子、前381）', '二十一年（庚子，公元前381年）\n\n'),
 ('二十二年（辛丑、前380）', '二十二年（辛丑，公元前380年）\n\n'),
 ('二十三年（壬寅、前379）', '二十三年（壬寅，公元前379年）\n\n'),
 ('二十四年（癸卯、前378）', '二十四年（癸卯，公元前378年）\n\n'),
 ('二十五年(甲辰、前377)', '二十五年（甲辰，公元前377年）\n\n'),
 ('二十六年（乙巳、前376）

Looks like '*' is '\n\n\u3000\u3000'.

So we replace it and check again:

In [6]:
re.findall(DUP_SUBTITLE_PAT, demo_f.replace('*', '\n\n\u3000\u3000'))

[('周纪一 威烈王二十三年（戊寅、前403）', '周纪一 周威烈王二十三年（戊寅，公元前403年）\n\n'),
 ('二十四年（己卯、前402）', '二十四年（己卯，公元前402年）\n\n'),
 ('安王元年（庚辰、前401）', '周安王元年（庚辰，公元前401年）\n\n'),
 ('二年（辛巳、前400）', '二年（辛巳，公元前400年）\n\n'),
 ('三年（壬午、前399）', '三年（壬午，公元前399年）\n\n'),
 ('四年（癸未、前398）', '四年（癸未，公元前398年）\n\n'),
 ('五年（甲申、前397）', '五年（甲申，公元前397年）\n\n'),
 ('六年（乙酉、前396）', '六年（乙酉，公元前396年）\n\n'),
 ('八年（丁亥、前394）', '八年（丁亥，公元前394年）\n\n'),
 ('九年（戊子、前393）', '九年（戊子，公元前393年）\n\n'),
 ('十一年（庚寅、前391）', '十一年（庚寅，公元前391年）\n\n'),
 ('十二年（辛卯、前390）', '十二年（辛卯，公元前390年）\n\n'),
 ('十三年（壬辰、前389）', '十三年（壬辰，公元前389年）\n\n'),
 ('十五年（甲午、前387）', '十五年（甲午，公元前387年）\n\n'),
 ('十六年（乙未、前386）', '十六年（乙未，公元前386年）\n\n'),
 ('十七年（丙申、前385）', '十七年（丙申，公元前385年）\n\n'),
 ('十九年（戊戌、前383）', '十九年（戊戌，公元前383年）\n\n'),
 ('二十年（己亥、前382）', '二十年（己亥，公元前382年）\n\n'),
 ('二十一年（庚子、前381）', '二十一年（庚子，公元前381年）\n\n'),
 ('二十二年（辛丑、前380）', '二十二年（辛丑，公元前380年）\n\n'),
 ('二十三年（壬寅、前379）', '二十三年（壬寅，公元前379年）\n\n'),
 ('二十四年（癸卯、前378）', '二十四年（癸卯，公元前378年）\n\n'),
 ('二十五年(甲辰、前377)', '二十五年（甲辰，公元前377年）\n\n'),
 ('二十六年（乙巳、前376）

All right.

In [7]:
demo_f = demo_f.replace('*', '\n\n\u3000\u3000')

让时间翻译对齐。

In [8]:
new_f = copy.copy(demo_f)
for g in re.findall(DUP_SUBTITLE_PAT, new_f):
    new_f = new_f.replace(g[1], g[1].lstrip('\u3000'))
    # print(g[1].lstrip('\u3000'))
re.findall(DUP_SUBTITLE_PAT, new_f)

[('周纪一 威烈王二十三年（戊寅、前403）', '周纪一 周威烈王二十三年（戊寅，公元前403年）\n\n'),
 ('二十四年（己卯、前402）', '二十四年（己卯，公元前402年）\n\n'),
 ('安王元年（庚辰、前401）', '周安王元年（庚辰，公元前401年）\n\n'),
 ('二年（辛巳、前400）', '二年（辛巳，公元前400年）\n\n'),
 ('三年（壬午、前399）', '三年（壬午，公元前399年）\n\n'),
 ('四年（癸未、前398）', '四年（癸未，公元前398年）\n\n'),
 ('五年（甲申、前397）', '五年（甲申，公元前397年）\n\n'),
 ('六年（乙酉、前396）', '六年（乙酉，公元前396年）\n\n'),
 ('八年（丁亥、前394）', '八年（丁亥，公元前394年）\n\n'),
 ('九年（戊子、前393）', '九年（戊子，公元前393年）\n\n'),
 ('十一年（庚寅、前391）', '十一年（庚寅，公元前391年）\n\n'),
 ('十二年（辛卯、前390）', '十二年（辛卯，公元前390年）\n\n'),
 ('十三年（壬辰、前389）', '十三年（壬辰，公元前389年）\n\n'),
 ('十五年（甲午、前387）', '十五年（甲午，公元前387年）\n\n'),
 ('十六年（乙未、前386）', '十六年（乙未，公元前386年）\n\n'),
 ('十七年（丙申、前385）', '十七年（丙申，公元前385年）\n\n'),
 ('十九年（戊戌、前383）', '十九年（戊戌，公元前383年）\n\n'),
 ('二十年（己亥、前382）', '二十年（己亥，公元前382年）\n\n'),
 ('二十一年（庚子、前381）', '二十一年（庚子，公元前381年）\n\n'),
 ('二十二年（辛丑、前380）', '二十二年（辛丑，公元前380年）\n\n'),
 ('二十三年（壬寅、前379）', '二十三年（壬寅，公元前379年）\n\n'),
 ('二十四年（癸卯、前378）', '二十四年（癸卯，公元前378年）\n\n'),
 ('二十五年(甲辰、前377)', '二十五年（甲辰，公元前377年）\n\n'),
 ('二十六年（乙巳、前376）

Looks nice.

Let's handle extra '\n' problem. **Except for the lines between title and text.**

In [9]:
re.findall('\n\n *\n+', new_f)

['\n\n\n\n']

In [10]:
help(re.sub)

Help on function sub in module re:

sub(pattern, repl, string, count=0, flags=0)
    Return the string obtained by replacing the leftmost
    non-overlapping occurrences of the pattern in string by the
    replacement repl.  repl can be either a string or a callable;
    if a string, backslash escapes in it are processed.  If it is
    a callable, it's passed the Match object and must return
    a replacement string to be used.



In [11]:
new_f = re.sub(re.compile(r'\n\n *\n+'), '\n\n', new_f)
new_f = new_f.replace('\n\n', '\n\n\n\n', 1).rstrip('\n')
print(new_f[:500])

资治通鉴第一卷(周纪)



周纪一 威烈王二十三年（戊寅、前403）

周纪一 周威烈王二十三年（戊寅，公元前403年）

　　[1]初命晋大夫魏斯、赵籍、韩虔为诸侯。

　　[1]周威烈王姬午初次分封晋国大夫魏斯、赵籍、韩虔为诸侯国君。

　　臣光曰：臣闻天子之职莫大于礼，礼莫大于分，分莫大于名。何谓礼？纪纲是也。何谓分？君、臣是也。何谓名？公、侯、卿、大夫是也。

　　臣司马光曰：我知道天子的职责中最重要的是维护礼教，礼教中最重要的是区分地位，区分地位中最重要的是匡正名分。什么是礼教？就是法纪。什么是区分地位？就是君臣有别。什么是名分？就是公、侯、卿、大夫等官爵。

　　夫以四海之广，兆民之众，受制于一人，虽有绝伦之力，高世之智，莫不奔走而服役者，岂非以礼为之纪纲哉！是故天子统三公，三公率诸侯，诸侯制卿大夫，卿大夫治士庶人。贵以临贱，贱以承贵。上之使下犹心腹之运手足，根本之制支叶，下之事上犹手足之卫心腹，支叶之庇本根，然后能上下相保而国家治安。故曰天子之职莫大于礼也。

　　四海之广，亿民之众，都受制于天子一人。尽管是才能超群、智慧绝伦的人，也不能不在天子足下为他奔走服务，这难道


Nice! 换行也修复了。但是还有下面的问题：
>　　夫礼，辨贵贱，序亲疏，裁群物，制庶事，非名不著，非器不形；名以命之，器以别之，然后上下粲然有伦，此礼之大经也。名器既亡，则礼安得独在哉！昔仲叔于奚有功于卫，辞邑而请繁缨，孔子以为不如多与之邑。惟名与器，不可以假人，君之所司也；政亡则国家从之。卫君待孔子而为政，孔子欲先正名，以为名不正则民无所措手足。夫繁缨，小物也，而孔子惜之；正名，细务也，而孔子先之：诚以名器既乱则上下无以相保故也。夫事未有不生于微而成于著，圣人之虑远，故能谨其微而治之，众人之识近，故必待其著而后救之；治其微则用力寡而功多，救其著则竭力而不能及也。《易》曰：“履霜坚冰至，”《书》曰：“一日二日万几，”谓此类也。故曰分莫大于名也。
>
> 　　所谓礼教，在于分辨贵贱，排比亲疏，裁决万物，处理日常事物。没有一定的名位，就不能显扬；没有器物，就不能表现。只有用名位来分别称呼，用器物来分别标志，然后上下才能井然有序。这就是礼教的根本所在。如果名位、器物都没有了，那么礼教又怎么能单独存在呢！当年仲叔于奚为卫国建立了大功，他谢绝了赏赐的封地，却请求允许他享用贵族才应有的马饰。孔子认为不如多赏赐他一些封地，惟独名位和器物，绝不能假与他人，这是君王的职权象征；处理政事不坚持原则，国家也就会随着走向危亡。卫国国君期待孔子为他崐处理政事，孔子却先要确立名位，认为名位不正则百姓无所是从。马饰，是一种小器物，而孔子却珍惜它的价值；正名位，是一件小事情，而孔子却要先从它做起，就是因为名位、器物一紊乱，国家上下就无法相安互保。没有一件事情不是从微小之处产生而逐渐发展显著的，圣贤考虑久远，所以能够谨慎对待微小的变故及时予以处理；常人见识短浅，所以必等弊端闹大才来设法挽救。矫正初起的小错，用力小而收效大；挽救已明显的大害，往往是竭尽了全力也
>
>　　不能成功。《易经》说：“行于霜上而知严寒冰冻将至。”《尚书》说：“先王每天都要兢兢业业地处理成千上万件事情。”就是指这类防微杜渐的例子。
>
>　　所以说，区分地位高下最重要的是匡正各个等级的名分。

其中属于一段的内容被错误换行了，尝试使用每段中的句号数量判断和修复这种情况。

In [12]:
# paras = new_f.split('\n\n')
# i = 2
# content = '\n\n'.join(paras[:i])

# while i < len(paras):
#     print(i)
#     p_raw = paras[i]
#     periods_cnt = p_raw.count('。')
#     i += 1
#     cnt = paras[i].count('。')
#     p_trans = [paras[i]]
#     while cnt < periods_cnt:
#         input()
#         print(p_raw)
#         print(p_trans)
#         i += 1
#         p_trans.append(paras[i].strip('\u3000'))
#         print(p_trans)
#         cnt = p_trans.count('。')
#         if cnt > periods_cnt:
#             raise RuntimeError()
#     content = '\n\n'.join([p_raw, ''.join(p_trans)])
#     i += 1

No way. '。' can't be the signal.

In [13]:
def try_fix_bad_para(f_content, debug=False):
    paras = f_content.split('\n\n')
    if '卷' in paras[0]:
        i = 2
        content = paras[0] + '\n' * 4
    else:
        i = 0
        content = ''
    new_paras = []

    try:
        while i < len(paras):
            p = paras[i].strip(' ')
            while p != '' and (len(re.findall('\d', p)) == 0) and p[-1] not in ['。', '？', '！', '”']:
                if debug:
                    input(p)
                i += 1
                p = p + paras[i].strip('\u3000')
                if debug:
                    print(p)
            new_paras.append(p)
            i += 1
    except Exception as e:
        return (i, paras)
    
    content += '\n\n'.join(new_paras)

    return 0, content

# try_fix_bad_para(new_f).split('\n\n')[:15]

Good! Edit all files!

In [14]:
!mkdir output

In [15]:
files = glob.glob('chapters/*.md')
pbar = tqdm(files, desc="Processing")
for f in pbar:
    pbar.set_description(f)
    content = open(f, 'r', encoding='utf-8').read()
    content = content.replace('*', '\n\n\u3000\u3000')
    for g in re.findall(DUP_SUBTITLE_PAT, content):
        if '[1]' in g[1]:
            continue
        content = content.replace(g[1], g[1].lstrip('\u3000'))
    content = re.sub('\n\n *\n+', '\n\n', content).rstrip('\n')
    if '卷' in content[:100]:
        content = content.replace('\n\n', '\n\n\n\n', 1)
    err_idx, content = try_fix_bad_para(content, debug=False)
    with open('output/' + f.split('/')[-1], 'w', encoding='utf-8') as wf:
        wf.write(content)

chapters/293_资治通鉴第二百九十三卷(后周纪).md: 100%|███████████████████████████████████████████████████████████████████████████████████████████| 294/294 [00:00<00:00, 914.79it/s]]s]]]?, ?it/s]
